# Import libraries, Load Data

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
#comment

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
import seaborn as sns
import yfinance as yf
from numpy import linalg as LA
from sklearn.cluster import KMeans
from statsmodels.tsa.stattools import coint
from itertools import combinations

In [47]:
# price_path = '/Users/tuckeringlefield/Desktop/FinanceData/price_data_from_shardar.csv'
# cap_path = '/Users/tuckeringlefield/Desktop/FinanceData/cap_data_from_shardar.csv'
price_path = "C:/Users/lukee/Downloads/price_data_from_shardar.csv"
cap_path = "C:/Users/lukee/Downloads/cap_data_from_shardar.csv"
prices_df = pd.read_csv(price_path, index_col='date')
caps_df = pd.read_csv(cap_path, index_col='date')
# Drop troubled stock
prices_df.drop(["MGI", "MDLZ", "DWA", "ICE"], axis=1, inplace=True)
caps_df.drop(["MGI", "MDLZ", "DWA", "ICE"], axis=1, inplace=True)

In [48]:
# Convert dates to datetime
prices_df.index = pd.to_datetime(prices_df.index)

# Get the initial start and end date
start_date = prices_df.index[0]
end_date = prices_df.index[-1]

spy_data = yf.download('SPY', start=start_date, end=end_date, interval='1d')
spy_data = pd.DataFrame(spy_data["Adj Close"])
spy_data.rename({"Adj Close": 'SPY'}, inplace=True, axis=1)
prices_with_market = prices_df.merge(spy_data, how='left', left_index=True, right_index=True)

df_diff = prices_with_market.diff().dropna()

[*********************100%%**********************]  1 of 1 completed


In [5]:
spy_data.isnull().sum()

SPY    0
dtype: int64

In [6]:
print(len(prices_with_market))
print(len(df_diff))

5787
0


# Outline and Function Definitions

Outline:

We want to start from the year 2000 and do our analysis in two periods:

- Learning Period

This will require 3 months of data. The top ten largest market cap stocks will be calculated. We should check for null values, this will let us know what stocks were recently added. We can then remove/add stocks by relevency. We then should calculate the correlation matrix for these stocks off of the last day of the three months. Then we will do k means clustering to find pairs that are cointegrated based of off the ADF.

- Testing Period

This will operate on two weeks of data. We will look at the spread between the stocks. If the spread goes more than 2 std deviations away from the mean for the correlation matrix period we will suggest opening a trade. Spread will come from the linear regression coefficient.

### Functions

In [7]:
# Function to find the top ten largest market cap stocks
def find_top_ten(dataframe, date_start, date_end):
    # Filter dataframe over the desired 3 months
    filtered_data = dataframe[date_start:date_end]
    target_date = date_end + 1
    selected_row = caps_df.iloc[target_date]
    selected_row_no_null = selected_row.dropna()
    stocks_list = selected_row_no_null.nlargest(10).index.tolist()
    return stocks_list

In [8]:
# Function to check for null values
def check_top_ten(dataframe, date_start, date_end, stocks_list):
    #should find the average and median ammount of null values per column
    #print this out and print out top ten col's ammounts
    ammount_null = []
    for column in df.columns:
        num_null = df[column].isnull().sum()
        ammount_null.append(num_null)
    average_num_null = np.mean(ammount_null)
    median_num_null = np.median(ammount_null)
    plt.boxplot(ammount_null, vert=False)
    plt.title('Boxplot of null values per stock in time period')
    plt.show()
    print("Checking Stocks List")
    print("--------------------")
    for column in stocks_list:
        num_null = df[column].isnull().sum()
        print(f'Stock: {column}, Num null: {num_null}')

In [36]:
# Function to filter the DF
def filter_diff_df(dataframe, date_start, date_end, stocks_list):
    desired_columns = stocks_list.copy()
    desired_columns.append("SPY")
    #print(desired_columns)
    filtered_df = dataframe[desired_columns]
    filtered_df = filtered_df[date_start:date_end].diff()#.dropna()
    print("Filtered diff df len (before dropna):", len(filtered_df))
    print(filtered_df.head())
    # print(len(filtered_df))
    # print(filtered_df.isnull().sum())
    filtered_df = filtered_df.dropna()
    print("Filtered diff df len (after dropna):", len(filtered_df))
    # print(len(filtered_df))
    return filtered_df

In [10]:
def filter_df_by_dates(dataframe, date_start, date_end, stocks_list):
    desired_columns = stocks_list.copy()
    desired_columns.append("SPY")
    filtered_df = dataframe[desired_columns]
    filtered_df = filtered_df[date_start:date_end]
    return filtered_df

In [28]:
# Function to calculate the betas
def calculate_betas(dataframe, date_start, date_end, stocks_list):
    # df = dataframe[date_start:date_end]
    beta_values = []
    columns = []

    df_var = dataframe['SPY'].var()

    for stk in stocks_list:
        df_cov = dataframe[[stk, 'SPY']].cov().loc[stk, 'SPY']
        beta = df_cov / df_var
        beta_values.append(beta)
        columns.append(stk + '_beta')

    beta_df = pd.DataFrame([beta_values], columns=columns)
    beta_df.index = dataframe.index[:1]

    # plt.figure(figsize=(12, 4))
    # sns.boxplot(data=beta_df)
    # plt.show()

    return beta_df

In [12]:
# Function to calculate the residuals
def calculate_residuals(df, stocks_list):
    res_df = pd.DataFrame()
    for stk in stocks_list:
        res_df[stk] = df[stk]-df[stk+"_beta"]*df["SPY"]
    return res_df

In [13]:
# Function to cluster the matrix
def cluster_the_matrix(df, num_clusters):
    A = abs(df.corr().values)
    #print(f'A Shape: {A.shape}')
    D = np.diag(A.sum(axis=1))
    #print(f'D Shape: {D.shape}')
    L = D - A
    #print(f'L Shape: {L.shape}')
    eigenvalues, eigenvectors = LA.eig(L)
    X = eigenvectors[:,:num_clusters]
    #print(f'X Shape: {X.shape}')
    kmeans = KMeans(n_clusters=num_clusters, random_state=2, n_init=20).fit(X)
    #print("Kmeans Labels:")
    #print(kmeans.labels_)
    #print(df.columns)

    cluster_dict = {}

    # Iterate over the indices of cluster_list
    for i in range(len(kmeans.labels_)):
        cluster_number = kmeans.labels_[i]
        stock_name = df.columns[i]

        # Check if cluster_number is already a key in the dictionary
        if cluster_number in cluster_dict:
            cluster_dict[cluster_number].append(stock_name)
        else:
            cluster_dict[cluster_number] = [stock_name]

    # fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    # scatter = ax.scatter(X[:, 0], X[:, 1], c=kmeans.labels_)
    # unique_labels = {label: idx for idx, label in enumerate(set(kmeans.labels_))}
    # handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=scatter.cmap(scatter.norm(value)), markersize=10)
    #        for value in unique_labels.values()]
    # labels = unique_labels.keys()
    # ax.legend(handles, labels, title="Clusters", loc="best", bbox_to_anchor=(1, 1))
    # ax.set_title(f'K-Means Clustering Results with K={num_clusters}')
    # plt.show()

    return cluster_dict


In [14]:
# Function to find the cointegrated pairs
def find_cointegrated_pairs(dataframe, cluster_dict, sig_level):
    cointegrated_pairs = []
    for cluster_num, stocks in cluster_dict.items():
      for stock1, stock2 in combinations(stocks, 2):
          pvalue1 = coint(dataframe[stock1], dataframe[stock2])[1]
          pvalue2 = coint(dataframe[stock2], dataframe[stock1])[1]
          if pvalue1 < sig_level and pvalue2 < sig_level:
              cointegrated_pairs.append((stock1, stock2))
    return cointegrated_pairs

In [15]:
# Function to check on existing pairs
def is_still_conintegrated(dataframe, pair, sig_level):
    stock1 = pair[0]
    stock2 = pair[1]
    pvalue1 = coint(dataframe[stock1], dataframe[stock2])[1]
    pvalue2 = coint(dataframe[stock2], dataframe[stock1])[1]
    if pvalue1 < sig_level and pvalue2 < sig_level:
        return True
    return False

In [16]:
# weekly function to calculate the beta of the pair
def calculate_beta_for_pair(dataframe, pair):
    asst1 = pair[0]
    asst2 = pair[1]

    train = dataframe[[asst1, asst2]]

    beta = train.cov().iloc[0, 1]/train[asst2].var()
    return beta

In [17]:
# Function to get the spread data
def get_spread_limits_for_past_months(dataframe, pair, beta):
    asst1 = pair[0]
    asst2 = pair[1]
    asst1_mean = dataframe[asst1].mean() 
    asst2_mean = dataframe[asst2].mean() 
    spread_data = None
    order = []
    if asst1_mean > asst2_mean:
        spread_data = dataframe[asst1]-beta*dataframe[asst2]
        order = [asst1, asst2]
    else:
        spread_data = dataframe[asst2]-beta*dataframe[asst1]
        order = [asst2, asst1]
    mean = spread_data.mean()
    std_dev = spread_data.std()
    lower_limit = mean - (2*std_dev)
    upper_limit = mean + (2*std_dev)
    
    return upper_limit, lower_limit, order
    

In [18]:
# Monitor a pair for a week

def monitor_pair_for_week(dataframe, start_date_index, end_date_index, curr_week_start_index, pair, pair_vals_list):
    #[ still_consecutive, [start_dates], [end_date], [[days_to_open_trade]] ]
    global prices_with_market
    start_date_string = prices_with_market.index[start_date_index].strftime('%Y-%m-%d')
    end_date_string = prices_with_market.index[end_date_index].strftime('%Y-%m-%d')
    curr_week_date_string = prices_with_market.index[curr_week_start_index].strftime('%Y-%m-%d')
    forward_three_months_data = prices_with_market[start_date_index+7 : end_date_index]

    if pair_vals_list[0] == True:
        past_three_month_data = prices_with_market[start_date_index : curr_week_start_index]
        beta_past_three_months = calculate_beta_for_pair(past_three_month_data, pair)
        upper, lower , pair_order = get_spread_limits_for_past_months(past_three_month_data, pair, beta_past_three_months)
        curr_week_data = dataframe.tail(7)
        curr_week_spread_data = curr_week_data[pair_order[0]]-beta_past_three_months*curr_week_data[pair_order[1]]
        for index, value in zip(curr_week_spread_data.index, curr_week_spread_data.values):
            date = index.strftime('%Y-%m-%d')
            if (value >= upper or value <= lower):
                pair_vals_list[-1][-1].append(date)
        
    
    is_coint = is_still_conintegrated(forward_three_months_data, pair , sig_level=0.05)
    if is_coint:
        if pair_vals_list[0] == False:
            pair_vals_list[1].append(curr_week_date_string)
            pair_vals_list[-1].append([])
            pair_vals_list[0] = True
    else:
        if pair_vals_list[0] == True:
            pair_vals_list[0] = False
            pair_vals_list[2].append(curr_week_date_string)

    return pair_vals_list

# start date ['2000-03-02', '2000-05-07']
# end date ['2000-03-28', '']
# trading days [ ['2000-03-15'], []]

In [34]:
def monitor_group_of_pairs(dataframe, pair_dict, stop_after_weeks):
    coint_dict = pair_dict.copy()
    weeks_running = 0
    while(weeks_running < stop_after_weeks):
        print(f"---------------------------- WEEKS RUNNING {weeks_running + 1} -------------------------------------")
        # On the first week we don't have any previous data in the pair_dict What do we pass in??? 
        start_date_index = ((weeks_running+1)*7)
        # always add 89 days here
        end_date_index = ((weeks_running+1)*7) + 97

        curr_week_start_index = ((weeks_running+1)*7) + 89
        # Filter the overall df
        three_month_plus_one_week_df = dataframe[start_date_index : end_date_index]
        for key, value in coint_dict.items():
            # end_date = start_date + WEEK            
            result = monitor_pair_for_week(three_month_plus_one_week_df, start_date_index, end_date_index, curr_week_start_index, key, value )
            # Update pair_dict[pair] with result
            coint_dict[key] = result
        # Prepping for the next week...
        
        # Finding the new cointegrated pairs
        new_top_ten_stocks = find_top_ten(prices_with_market, (start_date_index+7) , (end_date_index))
        # monitoring_data = filter_diff_df(prices_with_market, (start_date_index+7) , (end_date_index), new_top_ten_stocks)
        diff_df = filter_diff_df(prices_with_market, (start_date_index+7) , (end_date_index), new_top_ten_stocks)
        # print(f'Monitoring data: {len(monitoring_data)}')
        # filtered_monitoring_data = filter_df_by_dates(prices_with_market, (start_date_index+7) , end_date_index, new_top_ten_stocks)
        three_month_top_10_price_df = filter_df_by_dates(prices_with_market, (start_date_index+7) , end_date_index, new_top_ten_stocks)
        # print(f'Filtered Monitoring data: {len(filtered_monitoring_data)}')
        beta_df = calculate_betas(diff_df, (start_date_index+7) , end_date_index, new_top_ten_stocks)
        print(f'beta data: {len(beta_df)}')
        print("-------------------------")

        merged_df = diff_df.merge(beta_df, how = 'cross')
        res_df = calculate_residuals(merged_df, new_top_ten_stocks)
        cluster_dict = cluster_the_matrix(res_df, 5)
        pairs = find_cointegrated_pairs(three_month_top_10_price_df, cluster_dict, 0.05)
        #print(pairs)
        
        #found_pairs = find_contintegrated_pairs() 
        default_val_list_for_new_pair = [True, [dataframe.index[start_date_index]],[],[[]]]
        # for item in found_pairs:
            #if coint_dict[item] not in coint_dict:
                #otherwise add it as a new key with the default list
        weeks_running += 1
    return coint_dict

# Testing Functions

In [20]:
prices_with_market.head()

,ATW,A,AA,AAAB,AABC,AAC1,AACC,AACE,AACH,AADI,...,ZVOI,ZVRA,ZVUE,ZVXI,ZY,ZYME,ZYNE,ZYXI,ZZ,SPY
date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,8.766,44.275,NaN,8.036,7.791,4.562,NaN,17.000,NaN,NaN,...,NaN,NaN,NaN,3.625,NaN,NaN,NaN,NaN,NaN,93.290192
2000-01-04,8.913,40.893,NaN,7.619,7.791,4.438,NaN,15.875,NaN,NaN,...,NaN,NaN,NaN,3.500,NaN,NaN,NaN,NaN,NaN,89.641930
2000-01-05,8.899,37.855,NaN,7.500,7.791,4.438,NaN,16.125,NaN,NaN,...,NaN,NaN,NaN,3.708,NaN,NaN,NaN,NaN,NaN,89.802307
2000-01-06,9.270,36.896,NaN,7.500,7.791,4.500,NaN,16.125,NaN,NaN,...,NaN,NaN,NaN,3.833,NaN,NaN,NaN,NaN,NaN,88.359070
2000-01-07,9.508,39.971,NaN,7.738,7.607,4.562,NaN,16.750,NaN,NaN,...,NaN,NaN,NaN,4.167,NaN,NaN,NaN,NaN,NaN,93.490631


In [21]:
start = prices_with_market.index[0]
end = prices_with_market.index[89]
top_ten = find_top_ten(prices_with_market, start, end)

print(top_ten)
# check_top_ten(df, start, end, top_ten)
# print()

TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

In [ ]:
filtered_diff_df = filter_diff_df(prices_with_market, start, end, top_ten)
filtered_diff_df.head()
three_month_top_10_price_df = filter_df_by_dates(prices_with_market, start, end, top_ten)


In [ ]:
beta_df = calculate_betas(filtered_diff_df, start, end, top_ten)
print('Beta DF:')
print(beta_df)
print()

merged_df = filtered_diff_df.merge(beta_df, how = 'cross')

print('Residual DF:')
res_df = calculate_residuals(merged_df, top_ten)
print(res_df)

Beta DF:
            SCMR_beta  LVLT_beta  AMAT_beta  INTC_beta  CSCO_beta  JAVA1_beta  \
date                                                                            
2000-01-04  18.534859  16.697715   0.505715    0.47547   0.684213    2.472605   

            ORCL_beta  DELL1_beta  MSFT_beta  QCOM_beta  
date                                                     
2000-01-04   0.508758    0.451121   0.329183   0.901148  

Residual DF:
          SCMR       LVLT      AMAT      INTC      CSCO      JAVA1      ORCL  \
0    18.035173 -27.133127  0.701956  0.566617  0.393155  -0.479401 -0.275942   
1   -38.805433 -36.427810 -1.021101  0.409749  0.166273  -0.146531 -0.942589   
2   -69.082235 -22.850726  0.858881 -1.005770  0.022502  -4.181368 -0.802727   
3    13.219529 -54.635982 -2.411124 -1.504921 -1.477105  -4.938420 -1.093740   
4     9.055558  55.544761  1.589809  0.927509  1.124561  12.831994  2.377833   
..         ...        ...       ...       ...       ...        ...       ...   

In [ ]:
cluster_dictionary_for_top_10 = cluster_the_matrix(res_df, 4)

In [ ]:
coint_pairs_from_top_10 = find_cointegrated_pairs(three_month_top_10_price_df, cluster_dictionary_for_top_10, 0.05)
coint_pairs_from_top_10

[('INTC', 'ORCL')]

In [ ]:
pair_dict = {
        # [ still_consecutive, [start_dates], [end_date], times_correct, weeks_counted, curr_accuracy, [accuracy] ]
        ('INTC', 'ORCL') : [ True, ['2000-04-4'], [], 0, 0, 0, [] ]
    }

In [ ]:
three_month_top_10_price_df['ORCL']

date
2000-01-03    24.208
2000-01-04    22.076
2000-01-05    21.215
2000-01-06    19.678
2000-01-07    21.195
               ...  
2000-05-04    30.439
2000-05-05    31.488
2000-05-08    29.644
2000-05-09    29.517
2000-05-10    27.725
Name: ORCL, Length: 90, dtype: float64

In [ ]:
three_month_top_10_price_df['INTC']

date
2000-01-03    25.042
2000-01-04    23.874
2000-01-05    24.360
2000-01-06    22.668
2000-01-07    23.603
               ...  
2000-05-04    34.445
2000-05-05    35.539
2000-05-08    33.869
2000-05-09    33.667
2000-05-10    30.557
Name: INTC, Length: 90, dtype: float64

In [ ]:
asst1 = 'ORCL'
asst2 = 'INTC'

train = three_month_top_10_price_df[[asst1, asst2]]

beta = train.cov().iloc[0, 1]/train[asst2].var()
train_spread = train[asst1]-beta*train[asst2]
#test_spread = test[asst1]-beta*test[asst2]

print(train_spread)

date
2000-01-03   -0.123972
2000-01-04   -1.121089
2000-01-05   -2.454310
2000-01-06   -2.347284
2000-01-07   -1.738773
                ...   
2000-05-04   -3.029365
2000-05-05   -3.043346
2000-05-08   -3.264696
2000-05-09   -3.195424
2000-05-10   -1.965603
Length: 90, dtype: float64


In [ ]:
days_to_monitor = 7

whole_data = prices_with_market[[asst1,asst2]]
print(whole_data.head())
print()

date_to_start = whole_data.index[89]
for day in range(days_to_monitor):
    current_date = pd.to_datetime(whole_data.index[89 + day]).strftime("%Y-%m-%d")
    day_df = whole_data.loc[current_date]
    train_spread = day_df[asst1]-beta*day_df[asst2]
    print(train_spread)


              ORCL    INTC
date                      
2000-01-03  24.208  25.042
2000-01-04  22.076  23.874
2000-01-05  21.215  24.360
2000-01-06  19.678  22.668
2000-01-07  21.195  23.603

-1.9656030991385762
-2.6770280125542634
-1.76693316240042
-1.482613516084708
-1.6475948228379096
-2.4219622725933476
-4.720263225662265


In [49]:
pair_dict = {
        # [ still_consecutive, [start_dates], [end_date] weeks_counted, [[days_to_open_trade]] ]
        ('INTC', 'ORCL') : [ True, ['2000-04-04'], [], [] ]
    }
monitor_group_of_pairs(prices_with_market, pair_dict, 200)
# We know it's working right but we are corcerned about the is_coint function elimating possible trade days?



---------------------------- WEEKS RUNNING 1 -------------------------------------
Filtered diff df len (before dropna): 90
              SCMR  LVLT   CSCO   ORCL   QCOM  JAVA1   MSFT    VIAV   INTC  \
date                                                                         
2000-01-24     NaN   NaN    NaN    NaN    NaN    NaN    NaN     NaN    NaN   
2000-01-25  60.417 -60.0  1.034  0.923  2.878  7.000  0.485  36.401  0.803   
2000-01-26  19.791  -7.5 -1.689 -0.566 -7.890 -8.000 -1.069 -34.826 -1.468   
2000-01-27  43.959  22.5 -0.207 -1.332 -1.488 -3.375 -0.192 -10.710  0.469   
2000-01-28 -11.250 -74.1 -0.931 -1.816 -3.040 -3.750 -0.156 -36.949 -1.189   

              SDLI       SPY  
date                          
2000-01-24     NaN       NaN  
2000-01-25   5.719  1.022301  
2000-01-26  -4.188 -0.721619  
2000-01-27  18.938 -0.360786  
2000-01-28  -8.375 -2.806313  
Filtered diff df len (after dropna): 89
beta data: 1
-------------------------
---------------------------- WEEK

{('INTC', 'ORCL'): [False,
  ['2000-04-4', '2000-05-31', '2001-05-21', '2001-06-20', '2003-06-17'],
  ['2000-05-19', '2000-06-20', '2001-05-31', '2001-06-29', '2003-06-26'],
  [[],
   [],
   [],
   ['2003-06-30',
    '2003-07-01',
    '2003-07-02',
    '2003-07-03',
    '2003-07-07',
    '2003-07-08']]]}